# Generate Confusion Matrices

In [61]:
# Imports
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

%config InlineBackend.figure_format = 'svg'
np.set_printoptions(edgeitems=10,linewidth=180)

## Load the technology and its settings

Below, choose which technology to load data and settings for:

In [62]:
# Choose technology here
TECH = 'C'

# Load settings for technology
with open(f"../RRAM-Relaxation-Master-Model/data/tech{TECH}/settings.json") as sfile:
    settings = json.load(sfile)

In [63]:
# Load data for technology
colnames = ["addr", "time", "r", "g", "gi", "range", "timept"]
data = pd.read_csv(f"../RRAM-Relaxation-Master-Model/data/tech{TECH}/relaxdata.min.tsv.gz", names=colnames, sep='\t')
data.head()

,addr,time,r,g,gi,range,timept
0,10354,0.0,2.082326e+06,4.802322e-07,4.802322e-07,0,0.0
1,10191,0.0,2.375569e+06,4.209518e-07,4.209518e-07,0,0.0
2,13027,0.0,1.586060e+06,6.304932e-07,6.304932e-07,0,0.0
3,7397,0.0,2.592918e+06,3.856659e-07,3.856659e-07,0,0.0
4,2216,0.0,2.542619e+06,3.932953e-07,3.932953e-07,0,0.0


In [64]:
# Clip to [0, gmax]
data["g"] = np.clip(data["g"], 0, settings["gmax"]*0.99999999)
data["gi"] = np.clip(data["gi"], 0, settings["gmax"]*0.99999999)

# Get the initial and final range at each timept
data["irange"] = np.int16(data["gi"]/settings["gmax"]*32)
data["range"] = np.int16(data["g"]/settings["gmax"]*32)

In [65]:
# Show some data
data.head(20)

,addr,time,r,g,gi,range,timept,irange
0,10354,0.0,2.082326e+06,4.802322e-07,4.802322e-07,0,0.0,0
1,10191,0.0,2.375569e+06,4.209518e-07,4.209518e-07,0,0.0,0
2,13027,0.0,1.586060e+06,6.304932e-07,6.304932e-07,0,0.0,0
3,7397,0.0,2.592918e+06,3.856659e-07,3.856659e-07,0,0.0,0
4,2216,0.0,2.542619e+06,3.932953e-07,3.932953e-07,0,0.0,0
5,1721,0.0,9.517281e+05,1.050720e-06,1.050720e-06,0,0.0,0
6,4038,0.0,7.473174e+05,1.338120e-06,1.338120e-06,1,0.0,1
7,4431,0.0,8.672798e+05,1.153030e-06,1.153030e-06,0,0.0,0
8,11025,0.0,1.820318e+06,5.493546e-07,5.493546e-07,0,0.0,0
9,4822,0.0,1.634824e+06,6.116867e-07,6.116867e-07,0,0.0,0


In [66]:
# Get confusion matrix
for timept in settings["times"]:
    confmat = data[data["timept"] == timept].groupby(["range", "irange"]).size().unstack(fill_value=0)
    print(timept)
    print(confmat)
    np.save(f"confmats/tech{TECH}/{timept}.npy", confmat)

0
irange   0    1    2    3    4    5    6    7    8    9   ...   22   23   24  \
range                                                     ...                  
0       503    0    0    0    0    0    0    0    0    0  ...    0    0    0   
1         0  507    0    0    0    0    0    0    0    0  ...    0    0    0   
2         0    0  521    0    0    0    0    0    0    0  ...    0    0    0   
3         0    0    0  526    0    0    0    0    0    0  ...    0    0    0   
4         0    0    0    0  499    0    0    0    0    0  ...    0    0    0   
5         0    0    0    0    0  507    0    0    0    0  ...    0    0    0   
6         0    0    0    0    0    0  503    0    0    0  ...    0    0    0   
7         0    0    0    0    0    0    0  516    0    0  ...    0    0    0   
8         0    0    0    0    0    0    0    0  484    0  ...    0    0    0   
9         0    0    0    0    0    0    0    0    0  508  ...    0    0    0   
10        0    0    0    0    0    0  